In [ ]:
from fuzzywuzzy import fuzz, process
from simple_cysh import * # includes pandas as pd
import os
os.chdir(r'Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\Eval Management\ACM_analysis')

In [ ]:
# reference of commutes calculated at start of year
commutes_df = pd.read_csv('ACM_commutes_projected.csv', encoding='windows-1252')
# this table already contains matched-in commute info for some ACM names, which were cleaned manually in Excel
commutes_matched_df = pd.read_csv('ACM_commutes_projected_Q2survmatches.csv')
placements_df = pd.read_excel('SY18 FINAL Team Placements.xlsx')

In [ ]:
# load from Salesforce
staff_df = get_cysh_df('Staff__c', ['Id', 'Name', 'legacy_key2__c', 'Organization__c', 'Role__c'], "Site__c = 'Chicago'", rename_id=True)
staff_df.rename(columns={'Organization__c':'School__c', 'Role__c':'Title'}, inplace=True)
titles = ['Senior Corps Team Leader', 'Team Leader', 'Corps Member', 'Second Year Corps Member']
staff_df = staff_df.loc[staff_df['Title'].isin(titles)]
school_df = get_cysh_df('Account', ['Id', 'Name'])
school_df.rename(columns={'Id':'School__c', 'Name':'School_Name'}, inplace=True)

# Merge
acm_df = staff_df.merge(school_df, on='School__c', how='left')
acm_df = acm_df[['Staff__c', 'Name', 'Title', 'School_Name']]

In [ ]:
# Check that my match column is good, print non-matched rows
len(placements_df.loc[~placements_df['Full.Name (CYSH Staff Object)'].isin(acm_df['Name'])])

In [ ]:
acm_df = acm_df.merge(placements_df[['Full.Name (CYSH Staff Object)', 'Full.Name']], left_on='Name', right_on='Full.Name (CYSH Staff Object)', how='left')
del acm_df['Full.Name (CYSH Staff Object)']

In [ ]:
acm_df['Key'] = acm_df['Full.Name'] + "_" + acm_df['School_Name']
commutes_df['Key'] = commutes_df['Full.Name'] + "_" + commutes_df['School']
acm_df = acm_df.merge(commutes_df[['Key', 'Commute.Time']], on='Key', how='left')

In [ ]:
print(len(placements_df.loc[placements_df['Full.Name'].isin(commutes_df['Full.Name'])]))
print(f"{len(acm_df.loc[~acm_df['Commute.Time'].isnull()])} ACMs with commute calc")

In [ ]:
print(f"{len(acm_df.loc[~acm_df.Title.str.contains('Senior Corps Team Leader') & acm_df['Commute.Time'].isnull() & ~acm_df['Full.Name'].isnull()])} ACMs have no commute time calc")

In [ ]:
acm_df[['Staff__c', 'Commute.Time']].to_csv('commutes_cleaned.csv', index=False)